# Algoritmo de Tokenización Avanzada: BPE (Byte Pair Encoding)

En la sección anterior aprendimos a separar por espacios. Sin embargo, palabras como "like", "liker" y "lovely" comparten raíces. El algoritmo BPE busca pares de caracteres que aparecen juntos frecuentemente y los fusiona en un nuevo "token".

Esto permite que el modelo aprenda sílabas y morfemas en lugar de palabras enteras, lo cual es mucho más eficiente.

### 1. Preparación del Texto y Análisis de Caracteres
Primero, veamos qué caracteres únicos componen nuestro texto.

# Introducción a Byte Pair Encoding (BPE)

Hasta ahora, hemos tokenizado el texto dividiéndolo por **espacios** (palabras completas) o por **caracteres** (letras individuales). Sin embargo, los modelos de lenguaje modernos (como GPT-3, GPT-4 o Llama) utilizan un enfoque intermedio llamado **Subword Tokenization** (Tokenización por subpalabras), y el algoritmo más famoso para esto es **BPE (Byte Pair Encoding)**.

### ¿Cuál es el problema que resuelve?

1.  **Tokenización por palabras:** Si entrenas un modelo con la palabra "correr", y luego ve la palabra "corriendo", pensará que son dos cosas totalmente distintas si no están en su vocabulario. El diccionario se vuelve gigantesco e ineficiente.
2.  **Tokenización por caracteres:** Es muy flexible (puede escribir cualquier palabra), pero las secuencias se vuelven larguísimas y el modelo pierde el contexto del significado global de la palabra.

### La Solución BPE: La Estrategia de los "Legos"

Imagina que BPE es como jugar con Legos. En lugar de tener una pieza única para cada objeto del universo (una pieza "casa", una pieza "perro"), tienes bloques pequeños que se pueden unir.

* Al principio, BPE ve todo como piezas sueltas (letras individuales): `['c', 'o', 'r', 'r', 'e', 'r']`.
* Luego, nota que la **'r'** y la **'e'** aparecen juntas muy frecuentemente. Decide fusionarlas en una pieza nueva: **'re'**.
* Después, nota que **'er'** es muy común al final de las palabras. Crea el token **'er'**.

Al final, el modelo aprende que palabras como "correr", "comer" y "beber" comparten la terminación **"er"**.

In [1]:
# Texto con muchas repeticiones para probar el algoritmo
text = 'like liker love lovely hug hugs hugging hearts'

# Obtenemos los caracteres únicos convirtiendo a set (conjunto) y luego a lista
chars = list(set(text))
chars.sort() # Ordenamos alfabéticamente para mantener consistencia

print(f"Vocabulario inicial (caracteres): {chars}")

# Vemos la frecuencia de cada carácter
for l in chars:
    print(f'"{l}" aparece {text.count(l)} veces.')

Vocabulario inicial (caracteres): [' ', 'a', 'e', 'g', 'h', 'i', 'k', 'l', 'n', 'o', 'r', 's', 't', 'u', 'v', 'y']
" " aparece 7 veces.
"a" aparece 1 veces.
"e" aparece 5 veces.
"g" aparece 5 veces.
"h" aparece 4 veces.
"i" aparece 3 veces.
"k" aparece 2 veces.
"l" aparece 5 veces.
"n" aparece 1 veces.
"o" aparece 2 veces.
"r" aparece 2 veces.
"s" aparece 2 veces.
"t" aparece 1 veces.
"u" aparece 3 veces.
"v" aparece 2 veces.
"y" aparece 1 veces.


### 2. Crear el Vocabulario Inicial y la Lista de Tokens
El vocabulario inicial mapea cada carácter a un índice numérico (o simplemente lo guardamos como lista por ahora). También convertimos nuestro texto original en una lista de caracteres individuales.

In [2]:
# Creamos un vocabulario inicial (diccionario)
# Mapeamos el token (caracter) a su índice en la lista 'chars'
vocab = {char: idx for idx, char in enumerate(chars)}
print("Vocabulario Inicial:", vocab)

# El texto debe ser una lista mutable, no un string
# Cada elemento es un token
origtext = list(text)

print(f"\nTexto original (string): {text}")
print(f"Texto tokenizado (lista): {origtext}")

Vocabulario Inicial: {' ': 0, 'a': 1, 'e': 2, 'g': 3, 'h': 4, 'i': 5, 'k': 6, 'l': 7, 'n': 8, 'o': 9, 'r': 10, 's': 11, 't': 12, 'u': 13, 'v': 14, 'y': 15}

Texto original (string): like liker love lovely hug hugs hugging hearts
Texto tokenizado (lista): ['l', 'i', 'k', 'e', ' ', 'l', 'i', 'k', 'e', 'r', ' ', 'l', 'o', 'v', 'e', ' ', 'l', 'o', 'v', 'e', 'l', 'y', ' ', 'h', 'u', 'g', ' ', 'h', 'u', 'g', 's', ' ', 'h', 'u', 'g', 'g', 'i', 'n', 'g', ' ', 'h', 'e', 'a', 'r', 't', 's']


### 3. Definición de Funciones Principales

Para automatizar el proceso, necesitamos tres funciones clave:
1.  **`get_pair_stats`**: Cuenta la frecuencia de cada par de tokens adyacentes.
2.  **`update_vocab`**: Encuentra el par ganador y lo agrega al vocabulario.
3.  **`generate_new_token_seq`**: Reescribe el texto fusionando el par ganador.

In [3]:
# > -----------------------------------
def get_pair_stats(text2pair):
    """Cuenta cuántas veces aparece cada par de tokens consecutivos."""
    token_pairs = dict()

    # Recorremos los tokens hasta el penúltimo
    for i in range(len(text2pair)-1):

        # Creamos el par (tupla del token actual y el siguiente)
        pair = (text2pair[i], text2pair[i+1])

        # Incrementamos frecuencia
        if pair in token_pairs:
            token_pairs[pair] += 1
        else: # inicializar
            token_pairs[pair] = 1

    return token_pairs
# ----------------------------------- <

# > -----------------------------------
def update_vocab(token_pairs, vocab):
    """Encuentra el par más frecuente y lo añade al vocabulario."""

    # Encontrar el par con la máxima frecuencia
    # max() busca la clave con el valor más alto en el diccionario
    most_frequent_pair = max(token_pairs, key=token_pairs.get)

    # El nuevo token será la concatenación del par (ej: 'l' + 'i' = 'li')
    newtok = most_frequent_pair[0] + most_frequent_pair[1]

    # Actualizamos el vocabulario
    # Asignamos un nuevo ID (simplemente el tamaño actual del vocab + 1 o similar)
    next_id = len(vocab)
    vocab[newtok] = next_id

    print(f"--> Fusionando par: {most_frequent_pair} en nuevo token: '{newtok}'")

    # Devolvemos el vocabulario actualizado, el par que se fusionó y el nuevo token resultante
    return vocab, most_frequent_pair, newtok
# ----------------------------------- <

# > -----------------------------------
def generate_new_token_seq(prevtext, pair_to_merge, newtoken_str):
    """Reemplaza las ocurrencias del par separado por el nuevo token fusionado."""

    # Inicializar la nueva lista de texto
    newtext = []

    # Bucle a través de la lista
    i = 0
    while i < (len(prevtext)-1):

        # Probamos si el par de este elemento y el siguiente coinciden con el par ganador
        if prevtext[i] == pair_to_merge[0] and prevtext[i+1] == pair_to_merge[1]:
            # ¡Coincidencia! Añadimos el token fusionado
            newtext.append(newtoken_str)
            # Saltamos el siguiente carácter porque ya lo fusionamos
            i += 2

        # No es un par
        else:
            newtext.append(prevtext[i])
            # Nos movemos al siguiente carácter normalmente
            i += 1

    # El código original del video a veces omite el último carácter si no formaba parte de un par
    # Esta lógica asegura que si sobra un elemento al final, se agregue.
    if i == len(prevtext) - 1:
        newtext.append(prevtext[i])

    return newtext
# ----------------------------------- <

### 4. Ejercicio Principal: El Bucle de Entrenamiento
Ahora vamos a ejecutar el algoritmo iterativamente hasta alcanzar un tamaño de vocabulario deseado. Observa cómo la lista `updated_text` se va haciendo más corta a medida que los caracteres se fusionan en palabras.

In [4]:
# Re-inicializamos el vocabulario para asegurar limpieza
vocab = {char: idx for idx, char in enumerate(chars)}

# ¿Cuántos tokens queremos en total?
vocab_size = 25

# Hacemos una copia del texto original
updated_text = origtext.copy()

# Bucle principal
while len(vocab) < vocab_size:

    # 1. Obtener estadísticas de pares
    stats = get_pair_stats(updated_text)

    if not stats: # Si no hay más pares para fusionar, rompemos el ciclo
        break

    # 2. Actualizar el vocabulario con el mejor par
    # Nota: modifiqué update_vocab para devolver 3 valores para mayor claridad
    vocab, best_pair, new_token_str = update_vocab(stats, vocab)

    # 3. Generar la nueva secuencia de texto con el token fusionado
    updated_text = generate_new_token_seq(updated_text, best_pair, new_token_str)

print("\n--- Proceso Terminado ---")
print(f"Tamaño final del vocabulario: {len(vocab)}")
print(f"Texto final tokenizado: {updated_text}")

--> Fusionando par: (' ', 'h') en nuevo token: ' h'
--> Fusionando par: (' ', 'l') en nuevo token: ' l'
--> Fusionando par: (' h', 'u') en nuevo token: ' hu'
--> Fusionando par: (' hu', 'g') en nuevo token: ' hug'
--> Fusionando par: ('i', 'k') en nuevo token: 'ik'
--> Fusionando par: ('ik', 'e') en nuevo token: 'ike'
--> Fusionando par: (' l', 'o') en nuevo token: ' lo'
--> Fusionando par: (' lo', 'v') en nuevo token: ' lov'
--> Fusionando par: (' lov', 'e') en nuevo token: ' love'

--- Proceso Terminado ---
Tamaño final del vocabulario: 25
Texto final tokenizado: ['l', 'ike', ' l', 'ike', 'r', ' love', ' love', 'l', 'y', ' hug', ' hug', 's', ' hug', 'g', 'i', 'n', 'g', ' h', 'e', 'a', 'r', 't', 's']


### 5. Resultado Final
Observemos el vocabulario que ha aprendido la máquina. Debería contener palabras completas o partes significativas (como 'ing' o 'er') además de los caracteres individuales.

In [5]:
# Imprimir el vocabulario final ordenado por índice (si usamos índices secuenciales) o simplemente las claves
print("Vocabulario generado:")
for token, idx in vocab.items():
    print(f"Token: '{token}' \t ID: {idx}")

Vocabulario generado:
Token: ' ' 	 ID: 0
Token: 'a' 	 ID: 1
Token: 'e' 	 ID: 2
Token: 'g' 	 ID: 3
Token: 'h' 	 ID: 4
Token: 'i' 	 ID: 5
Token: 'k' 	 ID: 6
Token: 'l' 	 ID: 7
Token: 'n' 	 ID: 8
Token: 'o' 	 ID: 9
Token: 'r' 	 ID: 10
Token: 's' 	 ID: 11
Token: 't' 	 ID: 12
Token: 'u' 	 ID: 13
Token: 'v' 	 ID: 14
Token: 'y' 	 ID: 15
Token: ' h' 	 ID: 16
Token: ' l' 	 ID: 17
Token: ' hu' 	 ID: 18
Token: ' hug' 	 ID: 19
Token: 'ik' 	 ID: 20
Token: 'ike' 	 ID: 21
Token: ' lo' 	 ID: 22
Token: ' lov' 	 ID: 23
Token: ' love' 	 ID: 24


# 📉 ¿BPE reduce el vocabulario? La Paradoja de la Tokenización

Es común confundirse, pero la respuesta técnica es: **El vocabulario CRECE, pero la longitud del texto se REDUCE**.

Para entenderlo, debemos distinguir entre las dos dimensiones principales:

### 1. El Vocabulario (Tu caja de herramientas)
**Estado:** <span style="color:green">**AUMENTA 📈**</span>

El vocabulario es la lista de tokens únicos que tu modelo conoce.
* **Al inicio:** Tienes un vocabulario pequeño (solo letras individuales: `a, b, c...`).
* **Durante BPE:** El algoritmo *agrega* nuevos tokens fusionados (`er`, `ing`, `love`).
* **Resultado:** Terminas con **más tokens únicos** en tu diccionario que al principio.

### 2. La Secuencia (El mensaje a procesar)
**Estado:** <span style="color:red">**SE REDUCE 📉**</span>

Esto se refiere a la cantidad de tokens que la IA necesita leer para entender una frase (Compresión).
* **Antes (Caracteres):** Para leer **"like"**, la IA procesaba **4 tokens** (`['l', 'i', 'k', 'e']`).
* **Después (BPE):** Si aprendió el token **"like"**, ahora solo procesa **1 token** (`['like']`).

---

### 📊 Resumen Visual

| Concepto | Inicio (Solo Caracteres) | Final (Con BPE) | Resultado |
| :--- | :--- | :--- | :--- |
| **Vocabulario**<br>*(Piezas únicas)* | `['l', 'i', 'k', 'e'...]`<br>*(Pocas piezas, muy básicas)* | `['l', 'i', 'k', 'e', 'like'...]`<br>*(Más piezas, más complejas)* | **CRECE** (Más memoria necesaria) |
| **Secuencia**<br>*(Longitud del texto)* | `['l', 'i', 'k', 'e']`<br>*(4 pasos para leer)* | `['like']`<br>*(1 paso para leer)* | **SE REDUCE** (Más velocidad de cómputo) |

> **Conclusión:** Cambiamos un diccionario un poco más pesado a cambio de una lectura mucho más rápida y eficiente.